In [2]:
import pandas as pd
import numpy as np

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

pd.set_option('display.max_columns', 70)                       # 출력할 열의 최대개수
pd.set_option('display.max_colwidth', 80)                      # 출력할 열의 너비

In [5]:
review_matrix = pd.read_csv('리뷰 속 단어 체크체크.csv')
review_matrix

,id,place,count_review,avg_review,total_review,돈,쇼핑,시장,"싸다, 저렴하다",비싸다,먹다,"음식, 식사",상품,구매,가격,호텔,박물관,전시관,관광지,유적지,미술관,역사,유익하다,문화,궁전,전쟁,활동,놀이공원,게임,친구,커플,재미,아쿠아리움,엔터테인먼트,아이,가족,경기,힐링,공원,휴식,풍경,자연,전망대,산책,분수,여유,나들이,공연,콘서트,이벤트,영화,예술,미술,강좌,요리,공예,체험
0,1,교보문고 광화문,74,4.540540541,"['교보문고\n광화문역 5호선지하철역이랑 연결되어 있어서 찾아가기 아주좋아요 광화문교보문고 넓고 다양한 책들을 볼수있어 좋아오', '교보...",1.0,16.0,0.0,4.0,9.0,1.0,18.0,5.0,11.0,2.0,3.0,0.0,0.0,2.0,0.0,0.0,3.0,0.0,8.0,2.0,1.0,0.0,0.0,1.0,1.0,2.0,3.0,0.0,0.0,15.0,2.0,0.0,0.0,0.0,9.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,2.0,2.0,0.0,0.0,0.0,0.0,0.0
1,2,롯데월드타워&몰,203,4.206896552,"""좋은 쇼핑몰\n남자친구와 함께 방문한 곳이 있습니다. 이 많은 상품을 파는 브랜드 뿐만 아니라 다른 브랜드 인식 (하지만 더 저렴한)'...",0.0,1.0,0.0,1.0,2.0,0.0,2.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,3,SM타운 코엑스아티움,66,4.181818182,['뮤지컬 공연도 하고\n가끔 지나다가 들려서 구경할 때가 있는데 큰 기대는 안한다면 괜찮은 관광코스 되더라구요 지하철역에도 워낙 아이돌...,7.0,15.0,2.0,11.0,11.0,1.0,27.0,19.0,9.0,5.0,1.0,21.0,6.0,2.0,0.0,0.0,5.0,0.0,3.0,1.0,0.0,3.0,0.0,0.0,4.0,0.0,8.0,0.0,9.0,9.0,2.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,6.0,5.0,1.0,3.0,13.0,0.0,0.0,0.0,0.0,0.0
3,4,홍대앞 예술시장 프리마켓,75,4.386666667,['홍대앞 예술시장 프리마켓 이라고 되어있구나\n홍대라고 하면 여기까지 지칭해서 따로 없을줄 알았는데 있네요 여기 그냥 홍대앞에가서 거리...,3.0,46.0,23.0,18.0,4.0,14.0,52.0,7.0,2.0,20.0,3.0,1.0,0.0,6.0,0.0,1.0,0.0,1.0,2.0,0.0,0.0,1.0,0.0,0.0,4.0,1.0,9.0,0.0,0.0,6.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,6.0,0.0,0.0,2.0,14.0,0.0,0.0,0.0,10.0,1.0,0.0,0.0,6.0,0.0
4,5,광장시장,634,4.113564669,['생각보다는..\n워낙 사람이붐비고 유명한곳이라가봤는데 육회덮밥만먹고왔어요 생각보다 번잡스럽고 한복이나 의류상가들이 너무많아서 놀랐어요...,8.0,5.0,87.0,14.0,3.0,73.0,105.0,1.0,3.0,11.0,0.0,1.0,0.0,10.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,8.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,3.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1169,2297,디보 빌리지,1,1,"['폐쇄?\n우리는 우리가 그 문을 닫았다. 건물은 대부분의 상점이 큰 분명 우리는 거의 아무도, 귀타귀 종류의 문을 닫았습니다.']",0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1170,2300,서울빛초롱축제,36,4.666666667,"[""It's great art crafts. It is annual event every Autumn season.\n매년 열리는 빛초롱...",2.0,0.0,0.0,0.0,0.0,2.0,7.0,0.0,0.0,0.0,3.0,0.0,4.0,5.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,3.0,0.0,0.0,0.0,2.0,4.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,3.0,0.0,1.0,0.0,1.0,0.0
1171,2301,서울불꽃축제,22,4.363636364,"['불꽃놀이 구경\n사람이 무척 많습니다. 대중교통도 워낙 이용하기 어렵기에, 최대한 넉넉한 일정으로 가서 구경하고, 지하철로 이동하는 ...",0.0,0.0,0.0,2.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,2.0,0.0,2.0,2.0,1.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1172,2302,하이서울페스티벌,3,4.666666667,"['즐거운 서울!\n아침부터 저녁까지 정말 알찬 페스티벌입니다. 시청부터 종로, 명동까지 한번에 즐길수 있는 코스이구요, 다양한 부스 특...",0.0,0.0,1.0,0.0,0.0,1.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,1.0


In [6]:
tag_list = []

for i in range(len(review_matrix)):
    
    count_table = pd.DataFrame(review_matrix.iloc[i,5:])
    count_table = count_table.sort_values(by =[i], ascending = False)
    count_table = count_table[count_table[i] != 0]

    if len(count_table) >=7:
        count_table = count_table.iloc[:7,:]
        count_table_list = count_table.index.to_list()
    else : count_table_list = count_table.index.to_list()
    
    tag_list.append(count_table_list)
    
review_matrix['tag_list'] = tag_list
review_matrix

,id,place,count_review,avg_review,total_review,돈,쇼핑,시장,"싸다, 저렴하다",비싸다,먹다,"음식, 식사",상품,구매,가격,호텔,박물관,전시관,관광지,유적지,미술관,역사,유익하다,문화,궁전,전쟁,활동,놀이공원,게임,친구,커플,재미,아쿠아리움,엔터테인먼트,아이,가족,경기,힐링,공원,휴식,풍경,자연,전망대,산책,분수,여유,나들이,공연,콘서트,이벤트,영화,예술,미술,강좌,요리,공예,체험,tag_list
0,1,교보문고 광화문,74,4.540540541,"['교보문고\n광화문역 5호선지하철역이랑 연결되어 있어서 찾아가기 아주좋아요 광화문교보문고 넓고 다양한 책들을 볼수있어 좋아오', '교보...",1.0,16.0,0.0,4.0,9.0,1.0,18.0,5.0,11.0,2.0,3.0,0.0,0.0,2.0,0.0,0.0,3.0,0.0,8.0,2.0,1.0,0.0,0.0,1.0,1.0,2.0,3.0,0.0,0.0,15.0,2.0,0.0,0.0,0.0,9.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,2.0,2.0,0.0,0.0,0.0,0.0,0.0,"[음식, 식사, 쇼핑, 아이, 구매, 휴식, 비싸다, 문화]"
1,2,롯데월드타워&몰,203,4.206896552,"""좋은 쇼핑몰\n남자친구와 함께 방문한 곳이 있습니다. 이 많은 상품을 파는 브랜드 뿐만 아니라 다른 브랜드 인식 (하지만 더 저렴한)'...",0.0,1.0,0.0,1.0,2.0,0.0,2.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,"[상품, 비싸다, 음식, 식사, 싸다, 저렴하다, 쇼핑, 친구, 산책]"
2,3,SM타운 코엑스아티움,66,4.181818182,['뮤지컬 공연도 하고\n가끔 지나다가 들려서 구경할 때가 있는데 큰 기대는 안한다면 괜찮은 관광코스 되더라구요 지하철역에도 워낙 아이돌...,7.0,15.0,2.0,11.0,11.0,1.0,27.0,19.0,9.0,5.0,1.0,21.0,6.0,2.0,0.0,0.0,5.0,0.0,3.0,1.0,0.0,3.0,0.0,0.0,4.0,0.0,8.0,0.0,9.0,9.0,2.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,6.0,5.0,1.0,3.0,13.0,0.0,0.0,0.0,0.0,0.0,"[음식, 식사, 박물관, 상품, 쇼핑, 예술, 싸다, 저렴하다, 비싸다]"
3,4,홍대앞 예술시장 프리마켓,75,4.386666667,['홍대앞 예술시장 프리마켓 이라고 되어있구나\n홍대라고 하면 여기까지 지칭해서 따로 없을줄 알았는데 있네요 여기 그냥 홍대앞에가서 거리...,3.0,46.0,23.0,18.0,4.0,14.0,52.0,7.0,2.0,20.0,3.0,1.0,0.0,6.0,0.0,1.0,0.0,1.0,2.0,0.0,0.0,1.0,0.0,0.0,4.0,1.0,9.0,0.0,0.0,6.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,6.0,0.0,0.0,2.0,14.0,0.0,0.0,0.0,10.0,1.0,0.0,0.0,6.0,0.0,"[음식, 식사, 쇼핑, 시장, 가격, 싸다, 저렴하다, 먹다, 공연]"
4,5,광장시장,634,4.113564669,['생각보다는..\n워낙 사람이붐비고 유명한곳이라가봤는데 육회덮밥만먹고왔어요 생각보다 번잡스럽고 한복이나 의류상가들이 너무많아서 놀랐어요...,8.0,5.0,87.0,14.0,3.0,73.0,105.0,1.0,3.0,11.0,0.0,1.0,0.0,10.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,8.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,3.0,"[음식, 식사, 시장, 먹다, 싸다, 저렴하다, 가격, 관광지, 돈]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1169,2297,디보 빌리지,1,1,"['폐쇄?\n우리는 우리가 그 문을 닫았다. 건물은 대부분의 상점이 큰 분명 우리는 거의 아무도, 귀타귀 종류의 문을 닫았습니다.']",0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,[쇼핑]
1170,2300,서울빛초롱축제,36,4.666666667,"[""It's great art crafts. It is annual event every Autumn season.\n매년 열리는 빛초롱...",2.0,0.0,0.0,0.0,0.0,2.0,7.0,0.0,0.0,0.0,3.0,0.0,4.0,5.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,3.0,0.0,0.0,0.0,2.0,4.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,3.0,0.0,1.0,0.0,1.0,0.0,"[음식, 식사, 관광지, 재미, 전시관, 예술, 이벤트, 산책]"
1171,2301,서울불꽃축제,22,4.363636364,"['불꽃놀이 구경\n사람이 무척 많습니다. 대중교통도 워낙 이용하기 어렵기에, 최대한 넉넉한 일정으로 가서 구경하고, 지하철로 이동하는 ...",0.0,0.0,0.0,2.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,2.0,0.0,2.0,2.0,1.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,"[공원, 재미, 엔터테인먼트, 공연, 아이, 싸다, 저렴하다, 관광지]"
1172,2302,하이서울페스티벌,3,4.666666667,"['즐거운 서울!\n아침부터 저녁까지 정말 알찬 페스티벌입니다. 시청부터 종로, 명동까지 한번에 즐길수 있는 코스이구요, 다양한 부스 특...",0.0,0.0,1.0,0.0,0.0,1.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,1.0,"[예술, 공연, 음식, 식사, 체험, 커플, 문화, 아이]"


In [9]:
review_matrix.to_csv('리뷰 속 단어 체크체크 완료.csv', encoding='utf-8-sig', index=False)

PermissionError: [Errno 13] Permission denied: '리뷰 속 단어 체크체크 완료.csv'

In [7]:
review_matrix = pd.read_csv('data_for_service_1.csv')
review_matrix

,id,place,tag_list,count,avg_score,first_tag,second_tag,third_tag,fourth_tag,fifth_tag
0,1,교보문고 광화문,"['음식, 식사', '쇼핑', '아이', '구매', '휴식', '비싸다', '문화']",74,4.540541,"음식, 식사",쇼핑,아이,구매,구매
1,2,롯데월드타워&몰,"['상품', '비싸다', '음식, 식사', '싸다, 저렴하다', '쇼핑', '친구', '산책']",203,4.206897,상품,비싸다,"음식, 식사","싸다, 저렴하다","싸다, 저렴하다"
2,3,SM타운 코엑스아티움,"['음식, 식사', '박물관', '상품', '쇼핑', '예술', '싸다, 저렴하다', '비싸다']",66,4.181818,"음식, 식사",박물관,상품,쇼핑,쇼핑
3,4,홍대앞 예술시장 프리마켓,"['음식, 식사', '쇼핑', '시장', '가격', '싸다, 저렴하다', '먹다', '공연']",75,4.386667,"음식, 식사",쇼핑,시장,가격,가격
4,5,광장시장,"['음식, 식사', '시장', '먹다', '싸다, 저렴하다', '가격', '관광지', '돈']",634,4.113565,"음식, 식사",시장,먹다,"싸다, 저렴하다","싸다, 저렴하다"
...,...,...,...,...,...,...,...,...,...,...
1123,2297,디보 빌리지,['쇼핑'],1,1.000000,쇼핑,Nan,Nan,Nan,Nan
1124,2300,서울빛초롱축제,"['음식, 식사', '관광지', '재미', '전시관', '예술', '이벤트', '산책']",36,4.666667,"음식, 식사",관광지,재미,전시관,전시관
1125,2301,서울불꽃축제,"['공원', '재미', '엔터테인먼트', '공연', '아이', '싸다, 저렴하다', '관광지']",22,4.363636,공원,재미,엔터테인먼트,공연,공연
1126,2302,하이서울페스티벌,"['예술', '공연', '음식, 식사', '체험', '커플', '문화', '아이']",3,4.666667,예술,공연,"음식, 식사",체험,체험


In [8]:
cbf_matrix = review_matrix[['id','place','tag_list']]
cbf_matrix

,id,place,tag_list
0,1,교보문고 광화문,"['음식, 식사', '쇼핑', '아이', '구매', '휴식', '비싸다', '문화']"
1,2,롯데월드타워&몰,"['상품', '비싸다', '음식, 식사', '싸다, 저렴하다', '쇼핑', '친구', '산책']"
2,3,SM타운 코엑스아티움,"['음식, 식사', '박물관', '상품', '쇼핑', '예술', '싸다, 저렴하다', '비싸다']"
3,4,홍대앞 예술시장 프리마켓,"['음식, 식사', '쇼핑', '시장', '가격', '싸다, 저렴하다', '먹다', '공연']"
4,5,광장시장,"['음식, 식사', '시장', '먹다', '싸다, 저렴하다', '가격', '관광지', '돈']"
...,...,...,...
1123,2297,디보 빌리지,['쇼핑']
1124,2300,서울빛초롱축제,"['음식, 식사', '관광지', '재미', '전시관', '예술', '이벤트', '산책']"
1125,2301,서울불꽃축제,"['공원', '재미', '엔터테인먼트', '공연', '아이', '싸다, 저렴하다', '관광지']"
1126,2302,하이서울페스티벌,"['예술', '공연', '음식, 식사', '체험', '커플', '문화', '아이']"


In [9]:
from ast import literal_eval

listed_tag = []

for i in range (len(cbf_matrix)):
    tag = literal_eval(cbf_matrix.iloc[i,2])
    print(tag)
    print(type(tag))
    print(i)
    listed_tag.append(tag)

cbf_matrix['listed_tag'] = listed_tag
cbf_matrix

['음식, 식사', '쇼핑', '아이', '구매', '휴식', '비싸다', '문화']
<class 'list'>
0
['상품', '비싸다', '음식, 식사', '싸다, 저렴하다', '쇼핑', '친구', '산책']
<class 'list'>
1
['음식, 식사', '박물관', '상품', '쇼핑', '예술', '싸다, 저렴하다', '비싸다']
<class 'list'>
2
['음식, 식사', '쇼핑', '시장', '가격', '싸다, 저렴하다', '먹다', '공연']
<class 'list'>
3
['음식, 식사', '시장', '먹다', '싸다, 저렴하다', '가격', '관광지', '돈']
<class 'list'>
4
['쇼핑', '가격', '싸다, 저렴하다', '상품', '시장', '구매', '음식, 식사']
<class 'list'>
5
['쇼핑', '음식, 식사', '비싸다', '먹다', '상품', '시장', '싸다, 저렴하다']
<class 'list'>
6
['쇼핑', '음식, 식사', '영화', '비싸다', '시장', '먹다', '싸다, 저렴하다']
<class 'list'>
7
['음식, 식사', '쇼핑', '비싸다', '싸다, 저렴하다', '먹다', '관광지', '산책']
<class 'list'>
8
['쇼핑', '상품', '아이', '친구', '구매', '싸다, 저렴하다', '비싸다']
<class 'list'>
9
['쇼핑', '음식, 식사', '비싸다', '호텔', '싸다, 저렴하다', '아쿠아리움', '먹다']
<class 'list'>
10
['음식, 식사', '시장', '먹다', '재미', '싸다, 저렴하다', '구매', '돈']
<class 'list'>
11
['음식, 식사', '시장', '먹다', '가격', '싸다, 저렴하다', '쇼핑', '구매']
<class 'list'>
12
['시장', '음식, 식사', '먹다', '싸다, 저렴하다', '쇼핑', '상품', '가격']
<class 'list'>
13
['쇼핑', '음식, 식사

<class 'list'>
599
[]
<class 'list'>
600
['가족']
<class 'list'>
601
[]
<class 'list'>
602
['가족']
<class 'list'>
603
['쇼핑', '시장']
<class 'list'>
604
['호텔', '먹다']
<class 'list'>
605
['돈', '휴식']
<class 'list'>
606
['가격']
<class 'list'>
607
['친구', '힐링']
<class 'list'>
608
['친구']
<class 'list'>
609
['힐링', '체험', '가격', '커플', '친구', '호텔', '자연']
<class 'list'>
610
['친구']
<class 'list'>
611
['돈', '먹다', '이벤트']
<class 'list'>
612
['가격', '싸다, 저렴하다', '비싸다', '친구', '호텔']
<class 'list'>
613
['돈', '강좌']
<class 'list'>
614
['돈']
<class 'list'>
615
['궁전', '산책', '역사', '문화', '관광지', '친구', '자연']
<class 'list'>
616
['쇼핑', '관광지', '휴식', '산책', '싸다, 저렴하다', '궁전', '호텔']
<class 'list'>
617
['음식, 식사', '먹다', '휴식', '관광지', '궁전', '호텔', '쇼핑']
<class 'list'>
618
['역사', '공원', '산책', '먹다', '박물관', '자연', '풍경']
<class 'list'>
619
['궁전', '역사', '관광지', '문화', '산책', '공원', '쇼핑']
<class 'list'>
620
['쇼핑', '음식, 식사', '싸다, 저렴하다', '가격', '산책', '관광지', '먹다']
<class 'list'>
621
['쇼핑', '역사', '음식, 식사', '관광지', '휴식', '먹다', '호텔']
<class 'list'>
622
['

<ipython-input-9-2601c899969f>:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cbf_matrix['listed_tag'] = listed_tag


,id,place,tag_list,listed_tag
0,1,교보문고 광화문,"['음식, 식사', '쇼핑', '아이', '구매', '휴식', '비싸다', '문화']","[음식, 식사, 쇼핑, 아이, 구매, 휴식, 비싸다, 문화]"
1,2,롯데월드타워&몰,"['상품', '비싸다', '음식, 식사', '싸다, 저렴하다', '쇼핑', '친구', '산책']","[상품, 비싸다, 음식, 식사, 싸다, 저렴하다, 쇼핑, 친구, 산책]"
2,3,SM타운 코엑스아티움,"['음식, 식사', '박물관', '상품', '쇼핑', '예술', '싸다, 저렴하다', '비싸다']","[음식, 식사, 박물관, 상품, 쇼핑, 예술, 싸다, 저렴하다, 비싸다]"
3,4,홍대앞 예술시장 프리마켓,"['음식, 식사', '쇼핑', '시장', '가격', '싸다, 저렴하다', '먹다', '공연']","[음식, 식사, 쇼핑, 시장, 가격, 싸다, 저렴하다, 먹다, 공연]"
4,5,광장시장,"['음식, 식사', '시장', '먹다', '싸다, 저렴하다', '가격', '관광지', '돈']","[음식, 식사, 시장, 먹다, 싸다, 저렴하다, 가격, 관광지, 돈]"
...,...,...,...,...
1123,2297,디보 빌리지,['쇼핑'],[쇼핑]
1124,2300,서울빛초롱축제,"['음식, 식사', '관광지', '재미', '전시관', '예술', '이벤트', '산책']","[음식, 식사, 관광지, 재미, 전시관, 예술, 이벤트, 산책]"
1125,2301,서울불꽃축제,"['공원', '재미', '엔터테인먼트', '공연', '아이', '싸다, 저렴하다', '관광지']","[공원, 재미, 엔터테인먼트, 공연, 아이, 싸다, 저렴하다, 관광지]"
1126,2302,하이서울페스티벌,"['예술', '공연', '음식, 식사', '체험', '커플', '문화', '아이']","[예술, 공연, 음식, 식사, 체험, 커플, 문화, 아이]"


In [10]:
cbf_matrix.to_csv('cbf_matrix.csv', encoding= 'utf-8-sig', index=False)

In [11]:
cbf_matrix['tag_list_literal'] = cbf_matrix['listed_tag'].apply(lambda x : (' ').join(x))
cbf_matrix['tag_list_literal']

<ipython-input-11-8ee179dcec39>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cbf_matrix['tag_list_literal'] = cbf_matrix['listed_tag'].apply(lambda x : (' ').join(x))


0              음식, 식사 쇼핑 아이 구매 휴식 비싸다 문화
1        상품 비싸다 음식, 식사 싸다, 저렴하다 쇼핑 친구 산책
2       음식, 식사 박물관 상품 쇼핑 예술 싸다, 저렴하다 비싸다
3         음식, 식사 쇼핑 시장 가격 싸다, 저렴하다 먹다 공연
4         음식, 식사 시장 먹다 싸다, 저렴하다 가격 관광지 돈
                      ...               
1123                                  쇼핑
1124         음식, 식사 관광지 재미 전시관 예술 이벤트 산책
1125     공원 재미 엔터테인먼트 공연 아이 싸다, 저렴하다 관광지
1126            예술 공연 음식, 식사 체험 커플 문화 아이
1127                  문화 공원 역사 음식, 식사 공연
Name: tag_list_literal, Length: 1128, dtype: object

In [12]:
count_vect = CountVectorizer(min_df=0, ngram_range=(1,2))
tag_matrix = count_vect.fit_transform(cbf_matrix['tag_list_literal'])
print(tag_matrix.shape)

(1128, 1182)


In [13]:
tag_sim = cosine_similarity(tag_matrix, tag_matrix)
print(tag_sim.shape)
print(tag_sim[:5])

(1128, 1128)
[[1.         0.31311215 0.31311215 ... 0.06666667 0.33333333 0.31139958]
 [0.31311215 1.         0.52941176 ... 0.18786729 0.18786729 0.21938173]
 [0.31311215 0.52941176 1.         ... 0.18786729 0.25048972 0.21938173]
 [0.31311215 0.41176471 0.41176471 ... 0.25048972 0.25048972 0.29250897]
 [0.2        0.37573457 0.37573457 ... 0.26666667 0.2        0.23354968]]


In [14]:
tag_sim

array([[1.        , 0.31311215, 0.31311215, ..., 0.06666667, 0.33333333,
        0.31139958],
       [0.31311215, 1.        , 0.52941176, ..., 0.18786729, 0.18786729,
        0.21938173],
       [0.31311215, 0.52941176, 1.        , ..., 0.18786729, 0.25048972,
        0.21938173],
       ...,
       [0.06666667, 0.18786729, 0.18786729, ..., 1.        , 0.13333333,
        0.15569979],
       [0.33333333, 0.18786729, 0.25048972, ..., 0.13333333, 1.        ,
        0.38924947],
       [0.31139958, 0.21938173, 0.21938173, ..., 0.15569979, 0.38924947,
        1.        ]])

In [15]:
tag_sim_sorted_ind = tag_sim.argsort()[:, ::-1]

# 1번 장소[광화문 교보문고]와 유사도가 높은 인덱스는?
print(tag_sim_sorted_ind[3][:])

[  3  20 765 ... 966 400 382]


In [16]:
review_matrix['place'][797]

'신림동 순대타운'

In [17]:
코사인유사도_행렬 = pd.DataFrame(tag_sim)
코사인유사도_행렬.to_csv('그냥 코사인 유사도 행렬.csv')
코사인유사도_행렬

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,...,1093,1094,1095,1096,1097,1098,1099,1100,1101,1102,1103,1104,1105,1106,1107,1108,1109,1110,1111,1112,1113,1114,1115,1116,1117,1118,1119,1120,1121,1122,1123,1124,1125,1126,1127
0,1.000000,0.313112,0.313112,0.313112,0.200000,0.313112,0.313112,0.313112,0.375735,0.266667,0.313112,0.266667,0.313112,0.250490,0.313112,0.313112,0.313112,0.400000,0.313112,0.313112,0.250490,0.250490,0.333333,0.214834,0.375735,0.313112,0.233550,0.000000,0.429669,0.313112,0.313112,0.400000,0.266667,0.133333,0.429669,...,0.066667,0.155700,0.258199,0.097590,0.0,0.000000,0.000000,0.000000,0.115470,0.149071,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.346410,0.115470,0.286446,0.333333,0.266667,0.200000,0.233550,0.292770,0.0,0.172133,0.333333,0.071611,0.200000,0.214834,0.258199,0.200000,0.066667,0.333333,0.311400
1,0.313112,1.000000,0.529412,0.411765,0.375735,0.470588,0.529412,0.470588,0.529412,0.438357,0.470588,0.375735,0.470588,0.529412,0.411765,0.470588,0.470588,0.250490,0.470588,0.588235,0.470588,0.352941,0.313112,0.067267,0.470588,0.588235,0.219382,0.140028,0.336336,0.470588,0.529412,0.313112,0.250490,0.250490,0.403604,...,0.187867,0.073127,0.242536,0.091670,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.140028,0.325396,0.000000,0.134535,0.313112,0.187867,0.438357,0.073127,0.641689,0.0,0.000000,0.187867,0.067267,0.250490,0.470871,0.242536,0.250490,0.187867,0.187867,0.219382
2,0.313112,0.529412,1.000000,0.411765,0.375735,0.470588,0.529412,0.470588,0.470588,0.438357,0.470588,0.375735,0.411765,0.470588,0.470588,0.470588,0.470588,0.250490,0.470588,0.529412,0.470588,0.352941,0.313112,0.067267,0.411765,0.529412,0.219382,0.000000,0.336336,0.470588,0.470588,0.313112,0.187867,0.250490,0.336336,...,0.250490,0.073127,0.242536,0.091670,0.0,0.108465,0.280056,0.000000,0.000000,0.000000,0.140028,0.0,0.000000,0.000000,0.000000,0.000000,0.325396,0.000000,0.134535,0.313112,0.187867,0.375735,0.073127,0.550019,0.0,0.080845,0.187867,0.000000,0.250490,0.403604,0.242536,0.250490,0.187867,0.250490,0.219382
3,0.313112,0.411765,0.411765,1.000000,0.563602,0.588235,0.529412,0.529412,0.588235,0.250490,0.470588,0.500979,0.647059,0.588235,0.411765,0.470588,0.529412,0.438357,0.470588,0.529412,0.705882,0.529412,0.375735,0.134535,0.352941,0.588235,0.073127,0.000000,0.336336,0.529412,0.470588,0.313112,0.250490,0.375735,0.269069,...,0.375735,0.146254,0.323381,0.183340,0.0,0.108465,0.000000,0.242536,0.108465,0.140028,0.140028,0.0,0.140028,0.000000,0.000000,0.140028,0.433861,0.108465,0.067267,0.313112,0.187867,0.500979,0.000000,0.550019,0.0,0.000000,0.250490,0.067267,0.250490,0.538138,0.242536,0.187867,0.250490,0.250490,0.292509
4,0.200000,0.375735,0.375735,0.563602,1.000000,0.500979,0.500979,0.626224,0.500979,0.200000,0.438357,0.666667,0.688847,0.626224,0.375735,0.626224,0.563602,0.400000,0.438357,0.563602,0.563602,0.626224,0.266667,0.143223,0.375735,0.500979,0.077850,0.000000,0.358057,0.626224,0.563602,0.266667,0.266667,0.400000,0.214834,...,0.266667,0.077850,0.258199,0.097590,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.258199,0.000000,0.346410,0.000000,0.071611,0.266667,0.200000,0.466667,0.000000,0.585540,0.0,0.000000,0.266667,0.071611,0.266667,0.501280,0.000000,0.266667,0.266667,0.200000,0.233550
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1123,0.258199,0.242536,0.242536,0.242536,0.000000,0.242536,0.242536,0.242536,0.242536,0.258199,0.242536,0.000000,0.242536,0.242536,0.242536,0.242536,0.242536,0.258199,0.242536,0.242536,0.242536,0.000000,0.258199,0.277350,0.000000,0.242536,0.301511,0.000000,0.277350,0.242536,0.242536,0.258199,0.000000,0.000000,0.277350,...,0.000000

In [18]:
유사도_별로_장소_나열 = pd.DataFrame(tag_sim_sorted_ind)

In [21]:
유사도_별로_장소_나열[0] = list(review_matrix['place'])
유사도_별로_장소_나열.to_csv('유사도 순으로 장소 인덱스를 나열.csv')
유사도_별로_장소_나열 = 유사도_별로_장소_나열.iloc[:,:10]
유사도_별로_장소_나열.to_csv('유사도 순으로 장소 인덱스를 나열_10개만.csv', encoding = 'utf-8-sig', index=False)
유사도_별로_장소_나열

,0,1,2,3,4,5,6,7,8,9
0,교보문고 광화문,55,73,78,115,331,702,849,800,831
1,롯데월드타워&몰,1116,142,155,25,19,212,427,747,457
2,SM타운 코엑스아티움,155,212,457,747,39,142,1116,972,578
3,홍대앞 예술시장 프리마켓,20,765,642,12,643,136,653,44,648
4,광장시장,42,821,765,12,46,797,11,660,126
...,...,...,...,...,...,...,...,...,...,...
1123,디보 빌리지,183,1123,830,163,216,604,596,177,349
1124,서울빛초롱축제,132,173,638,994,751,200,566,514,583
1125,서울불꽃축제,1072,565,1121,839,493,509,1070,1025,259
1126,하이서울페스티벌,1109,109,873,336,485,200,849,176,958


In [62]:
list(review_matrix['place'])

['교보문고 광화문',
 '롯데월드타워&몰',
 'SM타운 코엑스아티움',
 '홍대앞 예술시장 프리마켓',
 '광장시장',
 " '시장 방문\\n광교와 장교 사이에 있다고 광장시장이라고 불리지만 이름 그대로 광장처럼 넓은 시장이기도 하다. 전통적인 한복 같은 것을 제외하더라도 한국적인 물건들을 파는 곳도 많다. 먹거리도 좋은데 특히 빈대떡과 육회가 일품이다.'",
 '고투 몰',
 '신세계 백화점 본점',
 '타임스퀘어',
 '가로수길',
 ' 동대문',
 '라인프렌즈 플래그십 스토어 명동역점',
 '스타필드 코엑스몰',
 ' 그것 은 아직 일부가 불완전하고 은 기본. 가장 중요한 것 - 방문해서 몰고 여름에는 일부 (특히 여전히 부품을 짓고 단장) 은 에어컨. 그 결과 내부에서 참을 몰 부족 환기와 질량 사람 불편 전체 경험',
 '통인시장',
 '마장 축산물시장',
 '남대문시장',
 " '길거리 음식 그리고 현지 시장\\n길거리 노점이 있고",
 '쌈지길',
 '롯데백화점 본점',
 '강남 지하상가',
 '중부시장',
 'IFC 몰',
 '동대문 쇼핑몰',
 '동대문 시장',
 '노량진 수산물 도매시장',
 " '가 지역\\n이야기 하며 그녀가 로컬 및 /. 다른 더 비싼 가격을 때 여러 사람들이 관광. 무엇을 선택할 수 있는 시푸드 레스토랑 에 무슨 요리 가 시장에 선택한 것입니다. 흥미로운 방문 이 처음 이지만 다시 갈 것이다. 바닥이 미끄러워서 시장 주의 와 함께 신발이 젖어 있다.'",
 '디큐브시브',
 '삼성 딜라이트',
 '카카오프렌즈 홍대 플래그십 스토어',
 '고속터미널상가',
 '신라면세점',
 'Unni Tour',
 '현대백화점 압구정본점',
 '커먼그라운드',
 '압구정 로데오 거리',
 '신세계 강남점',
 '프린세스다이어리 이대점',
 '경동시장',
 '라인 프렌즈 플래그십 스토어 가로수길점',
 '롯데아울렛 서울역점',
 '스타일난다 핑크호텔',
 '아이파크몰',
 '동대문 밀리오레',
 '롯데 영플라자 명동점',
 '서울약령

## tag 리스트 확인해서 사람의 interest 비율 알아보기

In [71]:
tagtag = pd.read_excel('tag.xlsx')
tagtag

,FLEX,교양인,놀래,쉴래,구경할래,배울래
0,돈,박물관,활동,힐링,공연,강좌
1,쇼핑,전시관,놀이공원,공원,콘서트,요리
2,시장,관광지,게임,휴식,이벤트,공예
3,"싸다, 저렴하다",유적지,친구,풍경,영화,체험
4,비싸다,미술관,커플,자연,예술,NaN
5,먹다,역사,재미,전망대,미술,NaN
6,"음식, 식사",유익하다,아쿠아리움,산책,NaN,NaN
7,상품,문화,엔터테인먼트,분수,NaN,NaN
8,구매,궁전,아이,여유,NaN,NaN
9,가격,전쟁,가족,나들이,NaN,NaN


## input 을 고려해 가상의 장소 선정

In [69]:
my_series = pd.Series({"여유":15, "힐링":10 , "책":5})
interest = pd.DataFrame(my_series, columns= ['정도'])
interest

,정도
여유,15
힐링,10
책,5


In [70]:
가능성_장소_리스트=
for i in range (len(review_matrix)):
    if '여유' in  review_matrix.iloc[i,-1]:
        
    

,id,place,count_review,avg_review,total_review,돈,쇼핑,시장,"싸다, 저렴하다",비싸다,먹다,"음식, 식사",상품,구매,가격,호텔,박물관,전시관,관광지,유적지,미술관,역사,유익하다,문화,궁전,전쟁,활동,놀이공원,게임,친구,커플,재미,아쿠아리움,엔터테인먼트,아이,가족,경기,힐링,공원,휴식,풍경,자연,전망대,산책,분수,여유,나들이,공연,콘서트,이벤트,영화,예술,미술,강좌,요리,공예,체험,tag_list
0,1,교보문고 광화문,74,4.540540541,"['교보문고\n광화문역 5호선지하철역이랑 연결되어 있어서 찾아가기 아주좋아요 광화문교보문고 넓고 다양한 책들을 볼수있어 좋아오', '교보...",1.0,16.0,0.0,4.0,9.0,1.0,18.0,5.0,11.0,2.0,3.0,0.0,0.0,2.0,0.0,0.0,3.0,0.0,8.0,2.0,1.0,0.0,0.0,1.0,1.0,2.0,3.0,0.0,0.0,15.0,2.0,0.0,0.0,0.0,9.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,2.0,2.0,0.0,0.0,0.0,0.0,0.0,"[음식, 식사, 쇼핑, 아이, 구매, 휴식, 비싸다, 문화]"
1,2,롯데월드타워&몰,203,4.206896552,"""좋은 쇼핑몰\n남자친구와 함께 방문한 곳이 있습니다. 이 많은 상품을 파는 브랜드 뿐만 아니라 다른 브랜드 인식 (하지만 더 저렴한)'...",0.0,1.0,0.0,1.0,2.0,0.0,2.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,"[상품, 비싸다, 음식, 식사, 싸다, 저렴하다, 쇼핑, 친구, 산책]"
2,3,SM타운 코엑스아티움,66,4.181818182,['뮤지컬 공연도 하고\n가끔 지나다가 들려서 구경할 때가 있는데 큰 기대는 안한다면 괜찮은 관광코스 되더라구요 지하철역에도 워낙 아이돌...,7.0,15.0,2.0,11.0,11.0,1.0,27.0,19.0,9.0,5.0,1.0,21.0,6.0,2.0,0.0,0.0,5.0,0.0,3.0,1.0,0.0,3.0,0.0,0.0,4.0,0.0,8.0,0.0,9.0,9.0,2.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,6.0,5.0,1.0,3.0,13.0,0.0,0.0,0.0,0.0,0.0,"[음식, 식사, 박물관, 상품, 쇼핑, 예술, 싸다, 저렴하다, 비싸다]"
3,4,홍대앞 예술시장 프리마켓,75,4.386666667,['홍대앞 예술시장 프리마켓 이라고 되어있구나\n홍대라고 하면 여기까지 지칭해서 따로 없을줄 알았는데 있네요 여기 그냥 홍대앞에가서 거리...,3.0,46.0,23.0,18.0,4.0,14.0,52.0,7.0,2.0,20.0,3.0,1.0,0.0,6.0,0.0,1.0,0.0,1.0,2.0,0.0,0.0,1.0,0.0,0.0,4.0,1.0,9.0,0.0,0.0,6.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,6.0,0.0,0.0,2.0,14.0,0.0,0.0,0.0,10.0,1.0,0.0,0.0,6.0,0.0,"[음식, 식사, 쇼핑, 시장, 가격, 싸다, 저렴하다, 먹다, 공연]"
4,5,광장시장,634,4.113564669,['생각보다는..\n워낙 사람이붐비고 유명한곳이라가봤는데 육회덮밥만먹고왔어요 생각보다 번잡스럽고 한복이나 의류상가들이 너무많아서 놀랐어요...,8.0,5.0,87.0,14.0,3.0,73.0,105.0,1.0,3.0,11.0,0.0,1.0,0.0,10.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,8.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,3.0,"[음식, 식사, 시장, 먹다, 싸다, 저렴하다, 가격, 관광지, 돈]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1169,2297,디보 빌리지,1,1,"['폐쇄?\n우리는 우리가 그 문을 닫았다. 건물은 대부분의 상점이 큰 분명 우리는 거의 아무도, 귀타귀 종류의 문을 닫았습니다.']",0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,[쇼핑]
1170,2300,서울빛초롱축제,36,4.666666667,"[""It's great art crafts. It is annual event every Autumn season.\n매년 열리는 빛초롱...",2.0,0.0,0.0,0.0,0.0,2.0,7.0,0.0,0.0,0.0,3.0,0.0,4.0,5.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,3.0,0.0,0.0,0.0,2.0,4.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,3.0,0.0,1.0,0.0,1.0,0.0,"[음식, 식사, 관광지, 재미, 전시관, 예술, 이벤트, 산책]"
1171,2301,서울불꽃축제,22,4.363636364,"['불꽃놀이 구경\n사람이 무척 많습니다. 대중교통도 워낙 이용하기 어렵기에, 최대한 넉넉한 일정으로 가서 구경하고, 지하철로 이동하는 ...",0.0,0.0,0.0,2.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,2.0,0.0,2.0,2.0,1.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,"[공원, 재미, 엔터테인먼트, 공연, 아이, 싸다, 저렴하다, 관광지]"
1172,2302,하이서울페스티벌,3,4.666666667,"['즐거운 서울!\n아침부터 저녁까지 정말 알찬 페스티벌입니다. 시청부터 종로, 명동까지 한번에 즐길수 있는 코스이구요, 다양한 부스 특...",0.0,0.0,1.0,0.0,0.0,1.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,1.0,"[예술, 공연, 음식, 식사, 체험, 커플, 문화, 아이]"
